In [1]:
#installing the libraries required
!pip install librosa

  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.0 MB 660.6 kB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.0 MB 550.5 kB/s eta 0:00:02
     ---- ----------------------------------- 0.1/1.0 MB 722.1 kB/s eta 0:00:02
     ------ --------------------------------- 0.2/1.0 MB 821.4 kB/s eta 0:00:

In [2]:
!pip install audiomentations

  Obtaining dependency information for audiomentations from https://files.pythonhosted.org/packages/0b/b6/dd82b8ade863bbe4c50fea7f06e696c306373a8dc042dd0aac0dc07bd857/audiomentations-0.34.1-py3-none-any.whl.metadata
  Using cached audiomentations-0.34.1-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/77.0 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/77.0 kB ? eta -:--:--
   --------------- ------------------------ 30.7/77.0 kB 330.3 kB/s eta 0:00:01
   ------------------------------------- -- 71.7/77.0 kB 563.7 kB/s eta 0:00:01
   ---------------------------------------- 77.0/77.0 kB 533.9 kB/s eta 0:00:00


In [70]:
!pip install pydub

In [74]:
!pip install lameenc

  Obtaining dependency information for lameenc from https://files.pythonhosted.org/packages/c8/5c/2ae2f48d32b954a93827469714506a45f9499df235a4e1c5c87849eb6550/lameenc-1.7.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/148.6 kB ? eta -:--:--
   -------- ------------------------------- 30.7/148.6 kB ? eta -:--:--
   ---------------- ---------------------- 61.4/148.6 kB 544.7 kB/s eta 0:00:01
   ------------------ -------------------- 71.7/148.6 kB 558.5 kB/s eta 0:00:01
   ------------------ -------------------- 71.7/148.6 kB 558.5 kB/s eta 0:00:01
   -------------------------------------- 148.6/148.6 kB 631.6 kB/s eta 0:00:00


In [1]:
import librosa
from audiomentations import Compose, PitchShift, AddGaussianNoise, HighPassFilter, LowPassFilter, Mp3Compression
from audiomentations import Padding, ClippingDistortion, TimeStretch, Gain

In [2]:
import soundfile as sf
from random import randint, choices
from glob import glob
import re

In [9]:
#the transformations can be changed here for future use
augument = Compose([
    PitchShift(min_semitones = -3, max_semitones = 3, p=1),
    HighPassFilter(min_cutoff_freq= 700, max_cutoff_freq= 1100, p=0.25),
    LowPassFilter(min_cutoff_freq= 2000, max_cutoff_freq= 2500, p=0.25),
    Padding(mode= "silence", max_fraction= 0.3, pad_section= choices(["start", "end"], weights= [1, 1])[0] , p=0.25),
    ClippingDistortion(max_percentile_threshold= randint(10, 20), p=0.4),
    TimeStretch(min_rate=0.97, max_rate= 1.45, leave_length_unchanged= False, p=0.5),
    #Mp3Compression(min_bitrate= 16, max_bitrate= 32, backend= "lameenc", p=1), 
        #the compression could be a better idea if higher quality audio is availible
    Gain(min_gain_db=7, max_gain_db=10, p=1),
    AddGaussianNoise(min_amplitude= 0.001, max_amplitude= 0.006, p=0.35)
])


In [11]:
#listing out the classes of audio
audio_types = ["belly_pain", "burping", "discomfort", "hungry", "tired"]

In [12]:

for audio_type in audio_types:
    #find the path to all the audio files in a class
    audio_files = glob(f"dataset1/{audio_type}/*.wav")
    for file in audio_files:
        
        #augumenting the files
        signal, sr = librosa.load(file)
        augumented = augument(signal, sr)
        
        #adding the augumented files back to the same class folders with the name [aug-(base file name).wav]
        file_str = re.search(r"\\(.+).wav", file)
        sf.write(f"dataset1/{audio_type}/aug-{file_str.group(1)}.wav", augumented, sr)